In [1]:
import pandas as pd
from ethtx import EthTx, EthTxConfig
from ethtx.models.decoded_model import DecodedTransaction
from typing import List

In [ ]:
txhash = '0x4c13a74b90bd9b6da0a648beebf355010b959ca714b3317461c0017b713fe354'

ethtx_config = EthTxConfig(
    mongo_connection_string="mongomock://localhost/ethtx",  ##MongoDB connection string,
    etherscan_api_key="VQXUYMB6T6I2E9SF67JPKP6YCYFW5P645A",  ##Etherscan API key,
    web3nodes={
        "mainnet": {
            "hook": "https://api.archivenode.io/58db800c-c787-41c1-96d5-ed35e126eb35",  # multiple nodes supported, separate them with comma
            "poa": True
        }
    },
    default_chain="mainnet",
    etherscan_urls={"mainnet": "https://api.etherscan.io/api", },
)

ethtx = EthTx.initialize(ethtx_config)
""" decoded_transaction: DecodedTransaction = ethtx.decoders.decode_transaction(
    '0x1594180567438ad258c98e263b412f55f62a23ebb5e9056089e9f11a513d4816')

print(decoded_transaction) """

In [ ]:
from ethtx.models.w3_model import W3Transaction, W3Block, W3Receipt, W3CallTree
web3provider = ethtx.providers.web3provider


# read raw transaction data directly from the node
w3transaction: W3Transaction = web3provider.get_transaction(txhash)
w3block: W3Block = web3provider.get_block(w3transaction.blockNumber)
w3receipt: W3Receipt = web3provider.get_receipt(w3transaction.hash.hex())
w3calls: W3CallTree = web3provider.get_calls(w3transaction.hash.hex())

print(w3transaction.dict())


In [ ]:
from ethtx.models.decoded_model import (
    DecodedTransfer,
    DecodedBalance,
    DecodedEvent, DecodedCall,
)
from ethtx.models.objects_model import Transaction, Event, Block, Call

# read the raw transaction from the node
transaction = Transaction.from_raw(
    w3transaction=w3transaction, w3receipt=w3receipt, w3calltree=w3calls
)

# get proxies used in the transaction
proxies = ethtx.decoders.get_proxies(transaction.root_call, "mainnet")

block: Block = Block.from_raw(
    w3block=web3provider.get_block(transaction.metadata.block_number),
    chain_id="mainnet",
)

# decode transaction components
abi_decoded_events: List[Event] = ethtx.decoders.abi_decoder.decode_events(
    transaction.events, block.metadata, transaction.metadata
)
abi_decoded_calls: DecodedCall = ethtx.decoders.abi_decoder.decode_calls(
    transaction.root_call, block.metadata, transaction.metadata, proxies
)
abi_decoded_transfers: List[
    DecodedTransfer
] = ethtx.decoders.abi_decoder.decode_transfers(abi_decoded_calls, abi_decoded_events)
abi_decoded_balances: List[DecodedBalance] = ethtx.decoders.abi_decoder.decode_balances(
    abi_decoded_transfers
)


In [ ]:

# decode a single event
raw_event: Event = transaction.events[0]
abi_decoded_event: DecodedEvent = ethtx.decoders.abi_decoder.decode_event(
    raw_event, block.metadata, transaction.metadata
)

# decode a single call
raw_call: Call = transaction.root_call.subcalls[0]
abi_decoded_call: DecodedCall = ethtx.decoders.abi_decoder.decode_call(
    raw_call, block.metadata, transaction.metadata, proxies
)


In [ ]:
from ethtx.models.decoded_model import DecodedTransactionMetadata

# semantically decode transaction components
decoded_metadata: DecodedTransactionMetadata = (
    ethtx.decoders.semantic_decoder.decode_metadata(
        block.metadata, transaction.metadata, "mainnet"
    )
)
decoded_events: List[DecodedEvent] = ethtx.decoders.semantic_decoder.decode_events(
    abi_decoded_events, decoded_metadata, proxies
)

decoded_calls: Call = ethtx.decoders.semantic_decoder.decode_calls(
    abi_decoded_calls, decoded_metadata, proxies
)
decoded_transfers: List[
    DecodedTransfer
] = ethtx.decoders.semantic_decoder.decode_transfers(
    abi_decoded_transfers, decoded_metadata
)
decoded_balances: List[
    DecodedBalance
] = ethtx.decoders.semantic_decoder.decode_balances(
    abi_decoded_balances, decoded_metadata
)


In [ ]:

""" # semantically decode a single event
decoded_event: DecodedEvent = ethtx.decoders.semantic_decoder.decode_event(
    abi_decoded_events[0], decoded_metadata, proxies
)
# semantically decode a single call
decoded_call: Call = ethtx.decoders.semantic_decoder.decode_call(
    abi_decoded_calls.subcalls[0], decoded_metadata, proxies
) """


In [ ]:
import sys
import json

sys.path.append("../src/")

from com.cryptobot.utils.logger import PrettyLogger

transaction_dict = transaction.dict()
abi_decoded_events_dict = [ev.dict() for ev in abi_decoded_events]
abi_decoded_calls_dict = decoded_calls.dict()
abi_decoded_transfers_dict = [t.dict() for t in abi_decoded_transfers]
abi_decoded_balances_dict = [b.dict() for b in abi_decoded_balances]

with open(f'../data/{txhash}_transaction.json', mode='a') as fp:
    json.dump(transaction_dict, fp, default=str)
    fp.close()

with open(f'../data/{txhash}_events.json', mode='a') as fp:
    json.dump(abi_decoded_events_dict, fp, default=str)
    fp.close()

with open(f'../data/{txhash}_calls.json', mode='a') as fp:
    json.dump(abi_decoded_calls_dict, fp, default=str)
    fp.close()

with open(f'../data/{txhash}_transfers.json', mode='a') as fp:
    json.dump(abi_decoded_transfers_dict, fp, default=str)
    fp.close()

with open(f'../data/{txhash}_balances.json', mode='a') as fp:
    json.dump(abi_decoded_balances_dict, fp, default=str)
    fp.close()


In [3]:
import sys
import json

sys.path.append("../src/")

from com.cryptobot.utils.path import get_data_path

tokens_holders_df = pd.read_csv(get_data_path() + 'tokens_holders.csv')

whale1 = tokens_holders_df[tokens_holders_df['address'] == '0xc882b111a75c0c657fc507c04fbfcd2cc984f071']
# whale1_usdt_qty = float('{0:.2f}'.format(float(whale1[0:1]['qty'])))
# (whale1_usdt_qty*100)/68291619089.0

whale1

,rank,address,qty,percentage,token_address,token_symbol
23,24,0xc882b111a75c0c657fc507c04fbfcd2cc984f071,1.023200e+08,2.531000e-01,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT
1325,76,0xc882b111a75c0c657fc507c04fbfcd2cc984f071,5.413214e+11,1.339076e+15,0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce,SHIB
3165,8,0xc882b111a75c0c657fc507c04fbfcd2cc984f071,6.644420e+04,1.643641e+08,0x2af5d2ad76741191d15dfe7bf6ac92d4bd912ca3,LEO
3722,65,0xc882b111a75c0c657fc507c04fbfcd2cc984f071,6.284298e+04,1.554557e+08,0x50d1c9771902476076ecfc8b2a83ad6b9355a4c9,FTT
3956,49,0xc882b111a75c0c657fc507c04fbfcd2cc984f071,1.667417e+07,4.124700e+00,0xa0b73e1ff0b80914ab6fe0444e65848c4c34450b,CRO
5892,235,0xc882b111a75c0c657fc507c04fbfcd2cc984f071,1.222412e+06,3.023902e+09,0x0f5d2fb29fb7d3cfee444a200298f468908cc942,MANA
9028,4,0xc882b111a75c0c657fc507c04fbfcd2cc984f071,2.977132e+07,7.364581e+10,0xe66747a101bff2dba3697199dcce5b743b454759,GT
9378,104,0xc882b111a75c0c657fc507c04fbfcd2cc984f071,1.834301e+06,4.537541e+09,0x4e15361fd6b4bb609fa63c81a2be19d873717870,FTM
10822,110,0xc882b111a75c0c657fc507c04fbfcd2cc984f071,3.988900e+05,9.867409e+08,0x1a4b46696b2bb4794eb3d4c26f1c55f9170fa4c5,BIT
12139,177,0xc882b111a75c0c657fc507c04fbfcd2cc984f071,2.343853e+04,5.798029e+07,0xc18360217d8f7ab5e7c516566761ea12ce7f9d72,ENS
